In [1]:
import pandas as pd
import json
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import autokeras as ak
import tensorflow as tf
import tensorflowjs as tfjs
import tf2onnx

Using TensorFlow backend


In [2]:
df = None
with open('../../src/data/master-search.json') as json_file:
    df = json.load(json_file)
rows = []
for k,v in df.items():
    rows.append(v)
df = pd.DataFrame(rows)
df.head()

,x,y,id,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,image_nsfw,prompt_nsfw,iaa_score
0,0.639129,-0.256337,ec9b5e2c-028e-48ac-8857-a52814fd2a06.png,"doom eternal, game concept art, veins and worm...",1,3312523387,50,7.0,3,512,512,596a88297105687d60162aaef5c077407170fda8ebf59f...,0.156454,0.008112,70.498428
1,-0.742775,6.939980,cd2a819b-faff-410a-af58-b371bd03c587.png,a beautiful photorealistic painting of cemeter...,1,3602562681,50,15.0,8,512,896,edbb4de830541f34a8f7d363bf8e375a4e65e6abed47ae...,0.084332,0.000207,69.245010
2,0.507401,-0.059566,baf9a21b-cdd0-4302-9fef-8435d7253981.png,"doom eternal, evangelion, game concept art, ve...",1,917080921,50,7.0,3,512,512,596a88297105687d60162aaef5c077407170fda8ebf59f...,0.534737,0.005789,74.754585
3,0.698210,-0.035485,d64b60b0-e214-4ab4-a7f6-0172460201f4.png,"doom eternal, metro exodus, game concept art, ...",1,1006826059,50,7.0,3,512,512,596a88297105687d60162aaef5c077407170fda8ebf59f...,0.266063,0.005960,72.161064
4,-0.496238,7.082948,c29ea381-ad6a-46b4-ada1-4224a7cbb676.png,"industrial architecture by zack snyder, atlant...",1,1609651727,50,15.0,8,512,896,edbb4de830541f34a8f7d363bf8e375a4e65e6abed47ae...,0.109691,0.000977,63.639427


In [3]:
df = df[['prompt', 'iaa_score']]
df.head()

,prompt,iaa_score
0,"doom eternal, game concept art, veins and worm...",70.498428
1,a beautiful photorealistic painting of cemeter...,69.245010
2,"doom eternal, evangelion, game concept art, ve...",74.754585
3,"doom eternal, metro exodus, game concept art, ...",72.161064
4,"industrial architecture by zack snyder, atlant...",63.639427


In [4]:
X_train,X_test,y_train,y_test=train_test_split(df['prompt'], df['iaa_score'], test_size=0.2)
X_train = np.array(X_train)
y_train = np.array(y_train)

In [5]:
vocab_size=1000
# Instantiate the tokenizer using the training data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df['prompt'])

In [6]:
def tokenize_to_ds(x_train):
    tokenized = tokenizer.texts_to_sequences(x_train)
    tokenized = tf.keras.preprocessing.sequence.pad_sequences(tokenized,maxlen=100)
    return tokenized

X_train_tokenized = tokenize_to_ds(X_train)
print(X_train_tokenized.shape)

(80000, 100)


In [7]:
vocab = {}
for word, index in tokenizer.word_index.items():
    if index <= vocab_size:
        vocab[word] = index
        
# Write the vocab dictionary to a json file for loading in javascript
with open('tokenizer_dictionary.json', 'w') as file:    
    json.dump(vocab, file)

In [8]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(100,)),
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [9]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.legacy.Adam(1e-4),
              metrics=['mean_absolute_error'])

In [11]:
history = model.fit(x=X_train_tokenized, y=y_train, epochs=10,
                    validation_split=0.2,
                    validation_steps=30)

Epoch 1/10
2000/2000 [==============================] - 58s 29ms/step - loss: 226.6132 - mean_absolute_error: 9.4365 - val_loss: 217.9935 - val_mean_absolute_error: 9.2663
Epoch 2/10
2000/2000 [==============================] - 57s 29ms/step - loss: 208.7591 - mean_absolute_error: 9.1303 - val_loss: 201.7051 - val_mean_absolute_error: 8.9671
Epoch 3/10
2000/2000 [==============================] - 61s 30ms/step - loss: 196.4622 - mean_absolute_error: 8.8682 - val_loss: 191.8030 - val_mean_absolute_error: 8.7925
Epoch 4/10
2000/2000 [==============================] - 59s 29ms/step - loss: 188.7776 - mean_absolute_error: 8.6928 - val_loss: 185.3565 - val_mean_absolute_error: 8.5775
Epoch 5/10
2000/2000 [==============================] - 55s 28ms/step - loss: 183.1721 - mean_absolute_error: 8.5597 - val_loss: 179.7248 - val_mean_absolute_error: 8.4328
Epoch 6/10
2000/2000 [==============================] - 56s 28ms/step - loss: 177.8111 - mean_absolute_error: 8.4170 - val_loss: 173.7526 - 

In [17]:
tfjs.converters.save_keras_model(model, 'Model_js')

/Users/justinblalock/miniconda3/envs/csefinalproject/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
